In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/cpc-codes/titles.csv
/kaggle/input/aigl-patentsberta-kaggle-v3/config.json
/kaggle/input/aigl-patentsberta-kaggle-v3/training_args.bin
/kaggle/input/aigl-patentsberta-kaggle-v3/tokenizer.json
/kaggle/input/aigl-patentsberta-kaggle-v3/tokenizer_config.json
/kaggle/input/aigl-patentsberta-kaggle-v3/pytorch_model.bin
/kaggle/input/aigl-patentsberta-kaggle-v3/special_tokens_map.json
/kaggle/input/aigl-patentsberta-kaggle-v3/vocab.txt
/kaggle/input/us-patent-phrase-to-phrase-matching/sample_submission.csv
/kaggle/input/us-patent-phrase-to-phrase-matching/train.csv
/kaggle/input/us-patent-phrase-to-phrase-matching/test.csv
/kaggle/input/debertav3small/spm.model
/kaggle/input/debertav3small/config.json
/kaggle/input/debertav3small/README.md
/kaggle/input/debertav3small/tf_model.h5
/kaggle/input/debertav3small/tokenizer_config.json
/kaggle/input/debertav3small/pytorch_model.bin


# Credits:
this notebook is based on the notbook ["Getting started with NLP for absolute beginners"](https://www.kaggle.com/code/jhoward/getting-started-with-nlp-for-absolute-beginners) by @Jeremy Howard

## Imports

In [2]:
import tensorflow as tf
import tensorflow_hub as hub
import seaborn as sns
import matplotlib.pyplot as plt

# EDA

In [3]:
train = pd.read_csv("/kaggle/input/us-patent-phrase-to-phrase-matching/train.csv")
print(train.shape)
train.head()

(36473, 5)


,id,anchor,target,context,score
0,37d61fd2272659b1,abatement,abatement of pollution,A47,0.50
1,7b9652b17b68b7a4,abatement,act of abating,A47,0.75
2,36d72442aefd8232,abatement,active catalyst,A47,0.25
3,5296b0c19e1ce60e,abatement,eliminating process,A47,0.50
4,54c1e3b9184cb5b6,abatement,forest region,A47,0.00


In [4]:
train.describe(include='object')

,id,anchor,target,context
count,36473,36473,36473,36473
unique,36473,733,29340,106
top,37d61fd2272659b1,component composite coating,composition,H01
freq,1,152,24,2186


In [5]:
train.groupby('score').count()

,id,anchor,target,context
score,,,,
0.00,7471,7471,7471,7471
0.25,11519,11519,11519,11519
0.50,12300,12300,12300,12300
0.75,4029,4029,4029,4029
1.00,1154,1154,1154,1154


## Score meanings according to [Data Description](https://www.kaggle.com/competitions/us-patent-phrase-to-phrase-matching/data?select=train.csv)

The scores are in the 0-1 range with increments of 0.25 with the following meanings:

    1.0 - Very close match. This is typically an exact match except possibly for differences in conjugation, quantity (e.g. singular vs. plural), and addition or removal of stopwords (e.g. “the”, “and”, “or”).
    0.75 - Close synonym, e.g. “mobile phone” vs. “cellphone”. This also includes abbreviations, e.g. "TCP" -> "transmission control protocol".
    0.5 - Synonyms which don’t have the same meaning (same function, same properties). This includes broad-narrow (hyponym) and narrow-broad (hypernym) matches.
    0.25 - Somewhat related, e.g. the two phrases are in the same high level domain but are not synonyms. This also includes antonyms.
    0.0 - Unrelated.


# Test Corpus

In [6]:
test = pd.read_csv("/kaggle/input/us-patent-phrase-to-phrase-matching/test.csv")

In [7]:
test.shape

(36, 4)

In [8]:
def cosine(u, v):
    """
    cosine similarity definition
    """
    return np.dot(u, v) / (np.linalg.norm(u) * np.linalg.norm(v))

# External Context

In [9]:
# ajouter le contexte https://www.cooperativepatentclassification.org/Archive
# extraire le contexte du fichier xml

In [10]:
titles = pd.read_csv("../input/cpc-codes/titles.csv")
print(titles.shape)
titles.head()

(260476, 7)


,code,title,section,class,subclass,group,main_group
0,A,HUMAN NECESSITIES,A,NaN,NaN,NaN,NaN
1,A01,AGRICULTURE; FORESTRY; ANIMAL HUSBANDRY; HUNTI...,A,1.0,NaN,NaN,NaN
2,A01B,SOIL WORKING IN AGRICULTURE OR FORESTRY; PARTS...,A,1.0,B,NaN,NaN
3,A01B1/00,Hand tools (edge trimmers for lawns A01G3/06 ...,A,1.0,B,1.0,0.0
4,A01B1/02,Spades; Shovels {(hand-operated dredgers E02F3...,A,1.0,B,1.0,2.0


# HuggingFace AutoModelForSequenceClassification

In [11]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from scipy.stats import pearsonr
from scipy.stats import spearmanr # pour trouver les correspondances entre les résultats sur des échelles variées
from sklearn.model_selection import StratifiedGroupKFold

In [12]:
import transformers
from transformers import TrainingArguments,Trainer
from transformers import AutoModelForSequenceClassification,AutoTokenizer

import torch
import datasets
from datasets import load_dataset, Dataset, DatasetDict

In [13]:
import warnings
import logging
from IPython.display import display, HTML

# Corpora merge

In [14]:
train['section'] = train["context"].str[0]
print(train.shape)
display(train.head())
titles=titles.rename(columns={"code": "context"})
display(titles.head())
train = train.merge(titles[["context", "title"]], on="context").rename(columns={"title": "context_title"})
train = train.merge(titles[["context", "title"]].rename(columns={"context": "section"}), on="section").rename(columns={"title": "section_title"})
display(train.head())

(36473, 6)


,id,anchor,target,context,score,section
0,37d61fd2272659b1,abatement,abatement of pollution,A47,0.50,A
1,7b9652b17b68b7a4,abatement,act of abating,A47,0.75,A
2,36d72442aefd8232,abatement,active catalyst,A47,0.25,A
3,5296b0c19e1ce60e,abatement,eliminating process,A47,0.50,A
4,54c1e3b9184cb5b6,abatement,forest region,A47,0.00,A


,context,title,section,class,subclass,group,main_group
0,A,HUMAN NECESSITIES,A,NaN,NaN,NaN,NaN
1,A01,AGRICULTURE; FORESTRY; ANIMAL HUSBANDRY; HUNTI...,A,1.0,NaN,NaN,NaN
2,A01B,SOIL WORKING IN AGRICULTURE OR FORESTRY; PARTS...,A,1.0,B,NaN,NaN
3,A01B1/00,Hand tools (edge trimmers for lawns A01G3/06 ...,A,1.0,B,1.0,0.0
4,A01B1/02,Spades; Shovels {(hand-operated dredgers E02F3...,A,1.0,B,1.0,2.0


,id,anchor,target,context,score,section,context_title,section_title
0,37d61fd2272659b1,abatement,abatement of pollution,A47,0.50,A,FURNITURE; DOMESTIC ARTICLES OR APPLIANCES; CO...,HUMAN NECESSITIES
1,7b9652b17b68b7a4,abatement,act of abating,A47,0.75,A,FURNITURE; DOMESTIC ARTICLES OR APPLIANCES; CO...,HUMAN NECESSITIES
2,36d72442aefd8232,abatement,active catalyst,A47,0.25,A,FURNITURE; DOMESTIC ARTICLES OR APPLIANCES; CO...,HUMAN NECESSITIES
3,5296b0c19e1ce60e,abatement,eliminating process,A47,0.50,A,FURNITURE; DOMESTIC ARTICLES OR APPLIANCES; CO...,HUMAN NECESSITIES
4,54c1e3b9184cb5b6,abatement,forest region,A47,0.00,A,FURNITURE; DOMESTIC ARTICLES OR APPLIANCES; CO...,HUMAN NECESSITIES


In [15]:
logging.disable(logging.WARNING)
warnings.simplefilter('ignore')

# PatentSBerta

In [16]:
#model_name = 'AI-Growth-Lab/PatentSBERTa_Kaggle_V3' #(online version)
model_name = '../input/aigl-patentsberta-kaggle-v3'

In [17]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [18]:
tokenizer.all_special_tokens

['<s>', '</s>', '[UNK]', '<pad>', '<mask>']

In [19]:
def clean(x):
    t = x.lower()
    t = t.replace("[",'')
    t = t.replace(";",'')
    t = t.replace(",",'')
    t = t.replace("]",'')
    t = t.replace(":",'')
    t = t.replace("(",'')
    t = t.replace(")",'')
    t = t.replace("{",'')
    t = t.replace("}",'')
    t = t.replace("/",' ')
    t = t.replace("-",' ')
    return t

In [20]:
train["inputs"] = train["section_title"].apply(clean) + " [SEP] " + train["anchor"] + " [SEP] " + train["target"]

In [21]:
def tok_func(x):
    return tokenizer(x["inputs"])

In [22]:
train_ds = Dataset.from_pandas(train).rename_column('score', 'label')

In [23]:
inps = "anchor","target","context","context_title","section_title"
tok_ds = train_ds.map(tok_func, batched=True, remove_columns=inps+('inputs','id','section'))

  0%|          | 0/37 [00:00<?, ?ba/s]

In [24]:
tok_ds

Dataset({
    features: ['label', '__index_level_0__', 'input_ids', 'attention_mask'],
    num_rows: 36473
})

In [25]:
# bs - batch size
lr,bs = 8e-5,64
# wd: weight decay
wd,epochs = 0.01,4

In [26]:
anchors = train["anchor"].unique()
np.random.shuffle(anchors)

In [27]:
val_prop = 0.25
val_sz = int(len(anchors)*val_prop)
val_anchors = anchors[:val_sz]

In [28]:
is_val = np.isin(train.anchor, val_anchors)
idxs = np.arange(len(train))
val_idxs = idxs[ is_val]
trn_idxs = idxs[~is_val]
len(val_idxs),len(trn_idxs)

(9927, 26546)

In [29]:
dds = DatasetDict({"train":tok_ds.select(trn_idxs),
             "test": tok_ds.select(val_idxs)})

In [30]:
def corr(eval_pred): 
    logits, labels = eval_pred
    logits = logits.reshape(-1)
    return {'pearson': np.corrcoef(logits, labels)[0][1]}

In [31]:
#args = TrainingArguments('outputs', learning_rate=lr, warmup_ratio=0.1, lr_scheduler_type='cosine', fp16=True,
    #evaluation_strategy="epoch", per_device_train_batch_size=bs, per_device_eval_batch_size=bs*2,
     #num_train_epochs=epochs, weight_decay=wd, report_to='none')
args = TrainingArguments('outputs', learning_rate=lr, warmup_ratio=0.1, lr_scheduler_type='cosine',
    evaluation_strategy="epoch", per_device_train_batch_size=bs, per_device_eval_batch_size=bs*2,
    num_train_epochs=epochs, weight_decay=wd, report_to='none')

In [32]:
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=1, ignore_mismatched_sizes=True)
trainer = Trainer(model, args, train_dataset=dds['train'], eval_dataset=dds['test'],
               tokenizer=tokenizer, compute_metrics=corr)

In [33]:
dds['train']

Dataset({
    features: ['label', '__index_level_0__', 'input_ids', 'attention_mask'],
    num_rows: 26546
})

In [34]:
eval = trainer.evaluate()

In [35]:
eval

{'eval_loss': 0.011976813897490501,
 'eval_pearson': 0.9171375891429188,
 'eval_runtime': 9.0952,
 'eval_samples_per_second': 1091.453,
 'eval_steps_per_second': 8.576}

In [36]:
trainer.num_examples(dds['train'])

1698944

In [37]:
#trainer.save_model()

# Test corpus preprocessing

In [38]:
test.head()

,id,anchor,target,context
0,4112d61851461f60,opc drum,inorganic photoconductor drum,G02
1,09e418c93a776564,adjust gas flow,altering gas flow,F23
2,36baf228038e314b,lower trunnion,lower locating,B60
3,1f37ead645e7f0c8,cap component,upper portion,D06
4,71a5b6ad068d531f,neural stimulation,artificial neural network,H04


In [39]:
test['section'] = test["context"].str[0]
print(test.shape)
display(titles.head())
test = test.merge(titles[["context", "title"]], on="context").rename(columns={"title": "context_title"})
test = test.merge(titles[["context", "title"]].rename(columns={"context": "section"}), on="section").rename(columns={"title": "section_title"})
display(test.head())

(36, 5)


,context,title,section,class,subclass,group,main_group
0,A,HUMAN NECESSITIES,A,NaN,NaN,NaN,NaN
1,A01,AGRICULTURE; FORESTRY; ANIMAL HUSBANDRY; HUNTI...,A,1.0,NaN,NaN,NaN
2,A01B,SOIL WORKING IN AGRICULTURE OR FORESTRY; PARTS...,A,1.0,B,NaN,NaN
3,A01B1/00,Hand tools (edge trimmers for lawns A01G3/06 ...,A,1.0,B,1.0,0.0
4,A01B1/02,Spades; Shovels {(hand-operated dredgers E02F3...,A,1.0,B,1.0,2.0


,id,anchor,target,context,section,context_title,section_title
0,4112d61851461f60,opc drum,inorganic photoconductor drum,G02,G,OPTICS,PHYSICS
1,5203a36c501f1b7c,generate in layer,generate by layer,G02,G,OPTICS,PHYSICS
2,7aa5908a77a7ec24,el display,illumination,G02,G,OPTICS,PHYSICS
3,442c114ed5c4e3c9,tunneling capacitor,capacitor housing,G11,G,INFORMATION STORAGE,PHYSICS
4,12ca31f018a2e2b9,speed control means,control loop,G05,G,CONTROLLING; REGULATING,PHYSICS


In [40]:
test["inputs"] = test["section_title"].apply(clean) + " [SEP] " + test["anchor"] + " [SEP] " + test["target"]

In [41]:
test_ds = Dataset.from_pandas(test)

In [42]:
test_ds

Dataset({
    features: ['id', 'anchor', 'target', 'context', 'section', 'context_title', 'section_title', 'inputs', '__index_level_0__'],
    num_rows: 36
})

In [43]:
inps = "anchor","target","context","context_title","section_title"
test_tok_ds = test_ds.map(tok_func, batched=True, remove_columns=inps+('inputs','id','section'))

  0%|          | 0/1 [00:00<?, ?ba/s]

In [44]:
preds_sberta = trainer.predict(test_tok_ds).predictions.astype(float)
preds_sberta

array([[ 0.48749384],
       [ 0.948901  ],
       [ 0.28694433],
       [ 0.50278115],
       [ 0.31292188],
       [ 0.73349011],
       [ 0.4781965 ],
       [ 0.79074466],
       [ 1.05603766],
       [ 0.2435392 ],
       [ 0.24521858],
       [ 0.55106753],
       [-0.01621631],
       [ 0.5375756 ],
       [ 0.63313073],
       [-0.00538462],
       [ 0.17454851],
       [ 0.79013216],
       [ 0.79604405],
       [-0.00391432],
       [ 0.27687433],
       [ 0.27295172],
       [ 0.52643836],
       [ 0.3617987 ],
       [ 0.01224874],
       [ 0.30376944],
       [ 0.29838696],
       [ 0.53494966],
       [ 0.12322883],
       [ 0.49000978],
       [ 0.48843858],
       [ 0.14339146],
       [-0.01222257],
       [ 0.1442931 ],
       [-0.0065867 ],
       [-0.00724098]])

In [45]:
type(preds_sberta)

numpy.ndarray

In [46]:
del model_name, tokenizer, model, tok_ds, test_tok_ds, dds

# Deberta-V3_small inference

In [47]:
#model_name = 'microsoft/deberta-v3-small' (online version)
model_name = '../input/debertav3small'

In [48]:
def corr_deberta(eval_pred): return {'pearson': np.corrcoef(*eval_pred)[0][1]}

In [49]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [50]:
tokenizer.all_special_tokens

['[CLS]', '[SEP]', '[UNK]', '[PAD]', '[MASK]']

In [51]:
tok_ds = train_ds.map(tok_func, batched=True, remove_columns=inps+('inputs','id','section'))

  0%|          | 0/37 [00:00<?, ?ba/s]

In [52]:
dds = DatasetDict({"train":tok_ds.select(trn_idxs),
             "test": tok_ds.select(val_idxs)})

In [53]:
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=1)
trainer = Trainer(model, args, train_dataset=dds['train'], eval_dataset=dds['test'],
               tokenizer=tokenizer, compute_metrics=corr_deberta)

In [54]:
trainer.train()

Epoch,Training Loss,Validation Loss,Pearson
1,No log,0.030505,0.786305
2,0.046300,0.024478,0.803951
3,0.018600,0.024581,0.805187
4,0.012200,0.024362,0.809510


TrainOutput(global_step=1660, training_loss=0.02423974246863859, metrics={'train_runtime': 261.3754, 'train_samples_per_second': 406.251, 'train_steps_per_second': 6.351, 'total_flos': 462650618918400.0, 'train_loss': 0.02423974246863859, 'epoch': 4.0})

In [55]:
eval_deberta = trainer.evaluate()
eval_deberta

{'eval_loss': 0.024362346157431602,
 'eval_pearson': 0.8095097173423833,
 'eval_runtime': 6.2466,
 'eval_samples_per_second': 1589.18,
 'eval_steps_per_second': 12.487,
 'epoch': 4.0}

In [56]:
test_tok_ds = test_ds.map(tok_func, batched=True, remove_columns=inps+('inputs','id','section'))

  0%|          | 0/1 [00:00<?, ?ba/s]

In [57]:
preds_deberta = trainer.predict(test_tok_ds).predictions.astype(float)
preds_deberta

array([[ 0.47512332],
       [ 0.93188006],
       [ 0.3357994 ],
       [ 0.42735329],
       [ 0.09561732],
       [ 0.75598294],
       [ 0.57309926],
       [ 0.73525792],
       [ 1.09592617],
       [ 0.22496426],
       [ 0.2182516 ],
       [ 0.5165168 ],
       [-0.01931833],
       [ 0.6032902 ],
       [ 0.54243577],
       [-0.01312365],
       [ 0.22481541],
       [ 0.79621786],
       [ 0.82313555],
       [-0.01335787],
       [ 0.27258927],
       [ 0.25757989],
       [-0.02062943],
       [ 0.27017459],
       [-0.01196046],
       [ 0.39508039],
       [ 0.43529677],
       [ 0.4923608 ],
       [ 0.28513709],
       [ 0.55472535],
       [ 0.45663142],
       [ 0.19141655],
       [-0.02379162],
       [ 0.25850114],
       [-0.02292352],
       [-0.00308337]])

In [58]:
del model_name, tokenizer, model

# Ensemble

ensemble methode inspired by Valentin Werner's "In Depth EDA & 3 Model Ensemble" notebook 

In [59]:
test_preds = 0.5 * preds_sberta + (0.5) * preds_deberta

In [60]:
test_preds

array([[ 4.81308579e-01],
       [ 9.40390527e-01],
       [ 3.11371863e-01],
       [ 4.65067223e-01],
       [ 2.04269599e-01],
       [ 7.44736522e-01],
       [ 5.25647879e-01],
       [ 7.63001293e-01],
       [ 1.07598191e+00],
       [ 2.34251730e-01],
       [ 2.31735088e-01],
       [ 5.33792168e-01],
       [-1.77673204e-02],
       [ 5.70432901e-01],
       [ 5.87783247e-01],
       [-9.25413705e-03],
       [ 1.99681960e-01],
       [ 7.93175012e-01],
       [ 8.09589803e-01],
       [-8.63609463e-03],
       [ 2.74731800e-01],
       [ 2.65265808e-01],
       [ 2.52904464e-01],
       [ 3.15986648e-01],
       [ 1.44139864e-04],
       [ 3.49424914e-01],
       [ 3.66841868e-01],
       [ 5.13655230e-01],
       [ 2.04182956e-01],
       [ 5.22367567e-01],
       [ 4.72534999e-01],
       [ 1.67404003e-01],
       [-1.80070982e-02],
       [ 2.01397121e-01],
       [-1.47551098e-02],
       [-5.16217644e-03]])

In [61]:
df_submission = pd.DataFrame()

In [62]:
df_submission['id'] = test['id'].copy()

In [63]:
df_submission

,id
0,4112d61851461f60
1,5203a36c501f1b7c
2,7aa5908a77a7ec24
3,442c114ed5c4e3c9
4,12ca31f018a2e2b9
5,9f2279ce667b21dc
6,b9ea2b06a878df6f
7,09e418c93a776564
8,ae0262c02566d2ce
9,03ba802ed4029e4d


In [64]:
preds_list = test_preds.tolist()

In [65]:
preds_list

[[0.48130857944488525],
 [0.9403905272483826],
 [0.3113718628883362],
 [0.46506722271442413],
 [0.20426959916949272],
 [0.744736522436142],
 [0.5256478786468506],
 [0.7630012929439545],
 [1.0759819149971008],
 [0.2342517301440239],
 [0.23173508793115616],
 [0.5337921679019928],
 [-0.01776732038706541],
 [0.5704329013824463],
 [0.5877832472324371],
 [-0.009254137054085732],
 [0.19968196004629135],
 [0.7931750118732452],
 [0.8095898032188416],
 [-0.008636094629764557],
 [0.2747317999601364],
 [0.2652658075094223],
 [0.2529044644907117],
 [0.31598664820194244],
 [0.000144139863550663],
 [0.34942491352558136],
 [0.3668418675661087],
 [0.5136552304029465],
 [0.2041829563677311],
 [0.5223675668239594],
 [0.47253499925136566],
 [0.16740400344133377],
 [-0.018007098231464624],
 [0.2013971209526062],
 [-0.014755109790712595],
 [-0.005162176443263888]]

In [66]:
flat_list = [x for xs in preds_list for x in xs]
flat_list

[0.48130857944488525,
 0.9403905272483826,
 0.3113718628883362,
 0.46506722271442413,
 0.20426959916949272,
 0.744736522436142,
 0.5256478786468506,
 0.7630012929439545,
 1.0759819149971008,
 0.2342517301440239,
 0.23173508793115616,
 0.5337921679019928,
 -0.01776732038706541,
 0.5704329013824463,
 0.5877832472324371,
 -0.009254137054085732,
 0.19968196004629135,
 0.7931750118732452,
 0.8095898032188416,
 -0.008636094629764557,
 0.2747317999601364,
 0.2652658075094223,
 0.2529044644907117,
 0.31598664820194244,
 0.000144139863550663,
 0.34942491352558136,
 0.3668418675661087,
 0.5136552304029465,
 0.2041829563677311,
 0.5223675668239594,
 0.47253499925136566,
 0.16740400344133377,
 -0.018007098231464624,
 0.2013971209526062,
 -0.014755109790712595,
 -0.005162176443263888]

In [67]:
df_submission['score' ] = flat_list

In [68]:
df_submission

,id,score
0,4112d61851461f60,0.481309
1,5203a36c501f1b7c,0.940391
2,7aa5908a77a7ec24,0.311372
3,442c114ed5c4e3c9,0.465067
4,12ca31f018a2e2b9,0.204270
5,9f2279ce667b21dc,0.744737
6,b9ea2b06a878df6f,0.525648
7,09e418c93a776564,0.763001
8,ae0262c02566d2ce,1.075982
9,03ba802ed4029e4d,0.234252


In [69]:
#submission = datasets.Dataset.from_dict({
  #  'id': test_ds['id'],
 #   'score': preds
#})

df_submission.to_csv('submission.csv', index=False)